<a href="https://colab.research.google.com/github/Srikanth1304/rep/blob/main/computation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np

def load_images_from_folder(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img = os.path.join(folder_path, filename)
        # if img is not None:
        #     img = cv2.resize(img, (227, 227))  # Resize the images to match the input shape of AlexNet
        #     img=np.array(img)
        images.append(img)
        labels.append(label)
    return images, labels

# Define the two folders containing the images and their corresponding labels
folder_path_classA = r"/content/drive/MyDrive/datset/datset/non_pcos"
folder_path_classB = r"/content/drive/MyDrive/datset/datset/pcos"

# Load images and labels for classA
images_classA, labels_classA = load_images_from_folder(folder_path_classA, label="non_pcos")

# Load images and labels for classB
images_classB, labels_classB = load_images_from_folder(folder_path_classB, label="pcos")


# Combine the data from both classes into a single DataFrame
data = {
    "image": images_classA + images_classB,
    "label": labels_classA + labels_classB
}

df = pd.DataFrame(data)

# Shuffle the DataFrame (optional)
df = df.sample(frac=1).reset_index(drop=True)

# Now, the DataFrame 'df' contains the image data and corresponding labels from both folders.
# You can use this DataFrame for further processing or training your model.


In [ ]:
from sklearn.preprocessing import LabelEncoder
# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the labels in the DataFrame
df["encoded_label"] = label_encoder.fit_transform(df["label"])

print(df)


                                                  image     label  \
0     /content/drive/MyDrive/datset/datset/pcos/17im...      pcos   
1     /content/drive/MyDrive/datset/datset/pcos/imag...      pcos   
2     /content/drive/MyDrive/datset/datset/pcos/imag...      pcos   
3     /content/drive/MyDrive/datset/datset/pcos/imag...      pcos   
4     /content/drive/MyDrive/datset/datset/pcos/imag...      pcos   
...                                                 ...       ...   
1042  /content/drive/MyDrive/datset/datset/pcos/1779...      pcos   
1043  /content/drive/MyDrive/datset/datset/pcos/imag...      pcos   
1044  /content/drive/MyDrive/datset/datset/pcos/imag...      pcos   
1045  /content/drive/MyDrive/datset/datset/pcos/imag...      pcos   
1046  /content/drive/MyDrive/datset/datset/non_pcos/...  non_pcos   

      encoded_label  
0                 1  
1                 1  
2                 1  
3                 1  
4                 1  
...             ...  
1042             

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

In [ ]:
def load_and_preprocess_image(image_path, image_width, image_height):
    image = Image.open(image_path)
    image = image.resize((image_width, image_height))
    image_array = np.array(image) / 255.0
    return image_array

In [ ]:
from tensorflow.keras.utils import to_categorical
X = np.array([load_and_preprocess_image(str(path), 224,224) for path in df['image']])
y = to_categorical(df['encoded_label'])

In [ ]:
# Split the dataset into training and testing sets
image_width = 224
image_height = 224
X = np.array([load_and_preprocess_image(str(path), image_width, image_height) for path in df['image']])


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_width, image_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(2, activation='softmax')  # Output layer with 2 units for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define a checkpoint callback to save the best model
checkpoint_path = "model_checkpoint.h5"
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path,
                                      save_best_only=True,
                                      verbose=1)

# Train the model
model.fit(X_train, y_train,
          epochs=30,
          batch_size=32,
          validation_split=0.2,
          callbacks=[checkpoint_callback])

# Load the best model from the checkpoint
model.load_weights(checkpoint_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Save the model as HDF5 file
model.save("dense_model.h5")

Epoch 1/30
20/21 [===========================>..] - ETA: 0s - loss: 0.4370 - accuracy: 0.8219
Epoch 1: val_loss improved from inf to 0.27362, saving model to model_checkpoint.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


21/21 [==============================] - 4s 101ms/step - loss: 0.4340 - accuracy: 0.8236 - val_loss: 0.2736 - val_accuracy: 0.9167
Epoch 2/30
20/21 [===========================>..] - ETA: 0s - loss: 0.1949 - accuracy: 0.9125
Epoch 2: val_loss improved from 0.27362 to 0.20849, saving model to model_checkpoint.h5
21/21 [==============================] - 2s 83ms/step - loss: 0.2090 - accuracy: 0.9103 - val_loss: 0.2085 - val_accuracy: 0.9405
Epoch 3/30
20/21 [===========================>..] - ETA: 0s - loss: 0.1701 - accuracy: 0.9219
Epoch 3: val_loss improved from 0.20849 to 0.12430, saving model to model_checkpoint.h5
21/21 [==============================] - 3s 171ms/step - loss: 0.1653 - accuracy: 0.9238 - val_loss: 0.1243 - val_accuracy: 0.9643
Epoch 4/30
21/21 [==============================] - ETA: 0s - loss: 0.1418 - accuracy: 0.9432
Epoch 4: val_loss did not improve from 0.12430
21/21 [==============================] - 1s 62ms/step - loss: 0.1418 - accuracy: 0.9432 - val_loss: 0.1

In [ ]:
from tensorflow.keras.models import load_model

# Load the model from the H5 file
loaded_model = load_model('/content/dense_model.h5')


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.optimizers import Adam
# Assuming you have a pre-trained feature extractor model (e.g., a CNN)
pretrained_feature_extractor = loaded_model.layers[-3]

# Get the output shape of the feature extraction layer
feature_shape = pretrained_feature_extractor.output_shape[1:]

base_model = models.Model(inputs=loaded_model.input, outputs=loaded_model.layers[-3].output)
# Print the shape for debugging
print("Feature Shape:", feature_shape)

# Define your transformer model
def transformer_model(input_shape, num_transformer_layers, output_dim):
    inputs = tf.keras.Input(shape=input_shape)

    # Add a "sequence_length" dimension to the input
    sequence = layers.Reshape((1, input_shape[0]))(inputs)

    # Transformer layers using MultiHeadAttention
    sequence = MultiHeadAttention(num_heads=5, key_dim=64)(sequence, sequence)
    sequence = layers.LayerNormalization(epsilon=1e-6)(sequence)
    sequence = layers.Dropout(0.1)(sequence)

    # Reshape the sequence to maintain 3D structure
    sequence = layers.Reshape((-1, feature_shape[0]))(sequence)

    # Output layer for your specific task
    outputs = layers.Dense(output_dim, activation='softmax')(sequence)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# Define your task-specific output dimension (e.g., number of classes)
output_dim = 2
num_tlayers = 2

# Create the transformer model
transformer = transformer_model(input_shape=feature_shape, num_transformer_layers=num_tlayers, output_dim=output_dim)

# Print the summary of the model for debugging
transformer.summary()

# Combine the pre-trained feature extractor with the transformer
combined_model = models.Sequential([
    base_model,
    transformer,
    layers.Flatten()  # Add Flatten layer to reshape the output
])

# Compile the model (adjust loss and metrics based on your task)
combined_model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# Print the summary of the combined model
combined_model.summary()


Feature Shape: (18432,)
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 18432)]              0         []                            
                                                                                                  
 reshape_2 (Reshape)         (None, 1, 18432)             0         ['input_2[0][0]']             
                                                                                                  
 multi_head_attention_1 (Mu  (None, 1, 18432)             2361235   ['reshape_2[0][0]',           
 ltiHeadAttention)                                        2          'reshape_2[0][0]']           
                                                                                                  
 layer_normalization_1 (Lay  (None, 1, 18432)             36864     

In [ ]:

# Train the combined model on your task-specific data
combined_model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
27/27 [==============================] - 6s 84ms/step - loss: 0.4670 - accuracy: 0.8961 - val_loss: 0.2272 - val_accuracy: 0.9286
Epoch 2/20
27/27 [==============================] - 2s 63ms/step - loss: 0.2040 - accuracy: 0.9379 - val_loss: 0.2370 - val_accuracy: 0.9381
Epoch 3/20
27/27 [==============================] - 2s 64ms/step - loss: 0.1458 - accuracy: 0.9510 - val_loss: 0.2435 - val_accuracy: 0.9286
Epoch 4/20
27/27 [==============================] - 2s 64ms/step - loss: 0.1459 - accuracy: 0.9606 - val_loss: 0.2450 - val_accuracy: 0.9524
Epoch 5/20
27/27 [==============================] - 2s 65ms/step - loss: 0.0851 - accuracy: 0.9713 - val_loss: 0.1907 - val_accuracy: 0.9524
Epoch 6/20
27/27 [==============================] - 2s 77ms/step - loss: 0.0675 - accuracy: 0.9785 - val_loss: 0.1906 - val_accuracy: 0.9571
Epoch 7/20
27/27 [==============================] - 2s 65ms/step - loss: 0.0585 - accuracy: 0.9773 - val_loss: 0.3008 - val_accuracy: 0.9429
Epoch 8/20
27

In [ ]:

# Train the combined model on your task-specific data
combined_model.fit(X,y, epochs=30, batch_size=32, validation_split=0.2)

Epoch 1/30
27/27 [==============================] - 2s 70ms/step - loss: 3.1956e-04 - accuracy: 1.0000 - val_loss: 0.5359 - val_accuracy: 0.9524
Epoch 2/30
27/27 [==============================] - 2s 63ms/step - loss: 3.1033e-04 - accuracy: 1.0000 - val_loss: 0.5357 - val_accuracy: 0.9571
Epoch 3/30
27/27 [==============================] - 2s 66ms/step - loss: 2.3028e-04 - accuracy: 1.0000 - val_loss: 0.5353 - val_accuracy: 0.9571
Epoch 4/30
27/27 [==============================] - 2s 69ms/step - loss: 2.4870e-04 - accuracy: 1.0000 - val_loss: 0.5347 - val_accuracy: 0.9524
Epoch 5/30
27/27 [==============================] - 2s 64ms/step - loss: 2.4608e-04 - accuracy: 1.0000 - val_loss: 0.5381 - val_accuracy: 0.9524
Epoch 6/30
27/27 [==============================] - 2s 64ms/step - loss: 1.9979e-04 - accuracy: 1.0000 - val_loss: 0.5424 - val_accuracy: 0.9571
Epoch 7/30
27/27 [==============================] - 2s 63ms/step - loss: 1.9042e-04 - accuracy: 1.0000 - val_loss: 0.5391 - val_ac

In [ ]:
import tensorflow as tf

# Check if TensorFlow is using GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Print GPU details
print("GPU Details:")
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    print(gpu)


Num GPUs Available:  1
GPU Details:
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [ ]:
combined_model.save('VIT.plt')

In [ ]:
import os
import pandas as pd
import numpy as np

def load_images_from_folder(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img = os.path.join(folder_path, filename)
        # if img is not None:
        #     img = cv2.resize(img, (227, 227))  # Resize the images to match the input shape of AlexNet
        #     img=np.array(img)
        images.append(img)
        labels.append(label)
    return images, labels

# Define the two folders containing the images and their corresponding labels
folder_path_classA = r"/content/drive/MyDrive/datset/datset/non_pcos"
folder_path_classB = r"/content/drive/MyDrive/datset/datset/pcos"

# Load images and labels for classA
images_classA, labels_classA = load_images_from_folder(folder_path_classA, label="pcos")

# Load images and labels for classB
images_classB, labels_classB = load_images_from_folder(folder_path_classB, label="non_pcos")


# Combine the data from both classes into a single DataFrame
data = {
    "image": images_classA + images_classB,
    "label": labels_classA + labels_classB
}

dft= pd.DataFrame(data)

# Shuffle the DataFrame (optional)
dft = df.sample(frac=1).reset_index(drop=True)

# Now, the DataFrame 'df' contains the image data and corresponding labels from both folders.
# You can use this DataFrame for further processing or training your model.


In [ ]:
from sklearn.preprocessing import LabelEncoder
# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the labels in the DataFrame
dft["encoded_label"] = label_encoder.fit_transform(dft["label"])

print(dft)


                                                  image     label  \
0     /content/drive/MyDrive/datset/datset/pcos/553i...      pcos   
1     /content/drive/MyDrive/datset/datset/non_pcos/...  non_pcos   
2     /content/drive/MyDrive/datset/datset/pcos/imag...      pcos   
3     /content/drive/MyDrive/datset/datset/non_pcos/...  non_pcos   
4     /content/drive/MyDrive/datset/datset/pcos/99im...      pcos   
...                                                 ...       ...   
1042  /content/drive/MyDrive/datset/datset/pcos/24im...      pcos   
1043  /content/drive/MyDrive/datset/datset/pcos/25im...      pcos   
1044  /content/drive/MyDrive/datset/datset/pcos/87im...      pcos   
1045  /content/drive/MyDrive/datset/datset/non_pcos/...  non_pcos   
1046  /content/drive/MyDrive/datset/datset/pcos/imag...      pcos   

      encoded_label  
0                 1  
1                 0  
2                 1  
3                 0  
4                 1  
...             ...  
1042             

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

In [ ]:
def load_and_preprocess_image(image_path, image_width, image_height):
    image = Image.open(image_path)
    image = image.resize((image_width, image_height))
    image_array = np.array(image) / 255.0
    return image_array

In [ ]:
from tensorflow.keras.utils import to_categorical
Xt = np.array([load_and_preprocess_image(str(path), 224,224) for path in dft['image']])
yt = to_categorical(dft['encoded_label'])

In [ ]:
import tensorflow as tf

# Load the model from the H5 file
loaded_model = load_model('VIT.plt')

# Get the output of the last convolutional layer
last_conv_layer_output = loaded_model.layers[-1].output  # Assuming the last convolutional layer is the fourth from the end

# Define a new model that outputs the features from the last convolutional layer
feature_extraction_model = tf.keras.Model(inputs=loaded_model.input, outputs=last_conv_layer_output)

# Use this model to extract features from your data
features = feature_extraction_model.predict(Xt)

# 'features' now contains the extracted features from the last convolutional layer
# You can use these features for further analysis or visualization


33/33 [==============================] - 1s 21ms/step


In [ ]:
print(features)

[[8.6729615e-06 9.9999130e-01]
 [1.0000000e+00 1.9862714e-17]
 [3.5304861e-06 9.9999642e-01]
 ...
 [1.0462336e-04 9.9989533e-01]
 [1.0000000e+00 1.0771153e-17]
 [5.4925398e-10 1.0000000e+00]]


In [ ]:
import tensorflow as tf

# Load the model from the H5 file
loaded_model = load_model('VIT.plt')

# Assuming Xt contains the input images with shape (None, 224, 224, 3)
# where None is the batch size and 224x224x3 is the image dimensions and channels
# Ensure that Xt has the correct shape
print("Input data shape:", Xt.shape)

# Use this model to extract features from your data
features = loaded_model.predict(Xt)

# 'features' now contains the extracted features
# You can use these features for further analysis or visualization


Input data shape: (1047, 224, 224, 3)
33/33 [==============================] - 1s 26ms/step


In [ ]:
from tensorflow.keras.models import load_model

# Load the model from the H5 file
model = load_model('VIT.plt')

# Now, you can use the loaded_model to make predictions on new data
# For example:
import numpy as np

# Assuming you have new data stored in the 'new_data' variable
predictions = model.predict(Xt)

# 'predictions' will contain the model's output for the new data
# You can now use 'predictions' for further processing or analysis


33/33 [==============================] - 1s 21ms/step


In [ ]:
thresholded_predictions = np.zeros_like(predictions)
thresholded_predictions[np.arange(len(predictions)), predictions.argmax(axis=1)] = 1

print(thresholded_predictions)

[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]


In [ ]:
from sklearn.metrics import classification_report
# Create a classification report
report = classification_report(yt,thresholded_predictions)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           0       0.99      0.95      0.97       144
           1       0.99      1.00      1.00       903

   micro avg       0.99      0.99      0.99      1047
   macro avg       0.99      0.97      0.98      1047
weighted avg       0.99      0.99      0.99      1047
 samples avg       0.99      0.99      0.99      1047



In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
# Create a classification report
report1 = multilabel_confusion_matrix(yt, thresholded_predictions)

# Print the classification report
print(report1)

[[[901   2]
  [  7 137]]

 [[137   7]
  [  2 901]]]


In [ ]:
# Calculate class-wise accuracy
class_accuracy = np.array([np.diag(conf_matrix) / np.sum(conf_matrix, axis=1) for conf_matrix in report1])

# Convert to percentage
class_accuracy_percentage = class_accuracy * 100

# Display class-wise accuracy
for i, acc in enumerate(class_accuracy_percentage):
    print(f"Class {i}: {acc[1]:.2f}%")  # Assuming the positive class is in the second row (index 1)

Class 0: 95.14%
Class 1: 99.78%


In [ ]:
# Extract true positives (TP) from the confusion matrix
TP = report1[:, 1, 1]  # Assuming positive class is 1, modify accordingly

# Calculate accuracy
accuracy = TP.sum() / len(yt)
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 99.14%
